In [ ]:
!pip install chainer
#!curl https://colab.chainer.org/install | sh -

In [2]:
import os
import argparse
import pickle
import pandas as pd
import networkx
import scipy
from time import time
import os
import argparse
import pickle
import pandas as pd
import networkx
import scipy
from time import time
from joblib import Parallel, delayed
import cupy as cp

In [8]:
from score_edges import generate_similarity_matrix, create_the_features_different_knockouts_iterative, create_the_features_different_knockouts
from glob_vars import SPECIES, PERT_MAP, TRAIN_DATA, HOME_DIR, LBL_DIR,\
    EDGES_DIR, FT_DIR, NET_DIR, PRT_DIR, NET_FILE, PROPAGATE_ALPHA,\
    PROPAGATE_EPSILON, PROPAGATE_ITERATIONS, N_JOBS

"""
Created on %(date)s

@author: L-F-S

SIGNAL
SIGN Annotation aLgorithm

Features generator

Inputs:
    Deletion signature dictionaries
    PPI interaction network
    Edges of interest

V4: Major refactoring. Edges input are now pkled files of lists of tuples.
 Network input is now pck of networkx.DiGraph (with all edges from training data included)

To update network, traning edges file, and perturbation dictionaries,
run preprocessing scripts inutility_scripts
"""


'\nCreated on %(date)s\n\n@author: L-F-S\n\nSIGNAL\nSIGN Annotation aLgorithm\n\nFeatures generator\n\nInputs:\n    Deletion signature dictionaries\n    PPI interaction network\n    Edges of interest\n\nV4: Major refactoring. Edges input are now pkled files of lists of tuples.\n Network input is now pck of networkx.DiGraph (with all edges from training data included)\n\nTo update network, traning edges file, and perturbation dictionaries,\nrun preprocessing scripts inutility_scripts\n'

In [5]:
print('------------------------------------------')
print('-------> SIGNAL features creation <-------')
print('------------------------------------------')
print()
print('>Inputs:')

# Load Network
with open(NET_DIR+NET_FILE, 'rb') as f:
    graph=pickle.load(f)
print('\t -Base network:', len(graph.nodes),'nodes;',len(graph.edges),'(directed) edges.')

# Load perturbation signatures:
with open(PRT_DIR+'plus_targets_'+PERT_MAP+'.pkl', 'rb') as f:
    plus_targets_of_deletion = pickle.load(f)
with open(PRT_DIR+'minus_targets_'+PERT_MAP+'.pkl', 'rb') as f:
    minus_targets_of_deletion = pickle.load(f)
print('\t -Perturbation signatures:\n\t\t- Positive knockout experiment targets:',\
      len(plus_targets_of_deletion),'\n\t\t- Negative knockout experiment targets:',len(minus_targets_of_deletion))

# Load Edges # Test only one dataset
TRAIN_DATA=['ubinet2']
edges_list=[]
for data in TRAIN_DATA:
    with open(EDGES_DIR+data+'.edges','rb') as f:
        edges=pickle.load(f) #list of edge names tuples
        edges_list+=[edges]
names=TRAIN_DATA
print('\t - Edges to create features for: ')
for name, edges in zip(names, edges_list):
    print('\t\t -', name, len(edges))
print('\t\t Total: ',sum([len(edges) for edges in edges_list]))
print('------------------------------------------')
print()

------------------------------------------
-------> SIGNAL features creation <-------
------------------------------------------

>Inputs:
	 -Base network: 5982 nodes; 303597 (directed) edges.
	 -Perturbation signatures:
		- Positive knockout experiment targets: 236 
		- Negative knockout experiment targets: 236
	 - Edges to create features for: 
		 - ubinet2 211
		 Total:  211
------------------------------------------



In [6]:
def generate_similarity_matrix_wrapper(graph):
    print('Generating similarity matrix..')
    genes = sorted(graph.nodes)
    raw_array = networkx.to_scipy_sparse_array(graph, genes, format='csc').T
    raw_matrix = scipy.sparse._csr.csr_matrix(raw_array)
    matrix, raw_col_sums = generate_similarity_matrix(raw_matrix) #normalized similarity matrix
    num_genes     = len(genes)
    gene_indexes  = dict([(gene, index) for (index, gene) in enumerate(genes)]) #all genes present in matrix

    return genes, raw_matrix, matrix, raw_col_sums, num_genes, gene_indexes

genes, raw_matrix, matrix, raw_col_sums, num_genes, gene_indexes = generate_similarity_matrix_wrapper(graph)

Generating similarity matrix..


In [9]:
print('Number of threads:', N_JOBS)
for name, edges in zip(names, edges_list):
    start=time()
    packed_results = Parallel(n_jobs=N_JOBS)(delayed(create_the_features_different_knockouts)\
                                (raw_matrix, edge, gene_indexes, matrix, plus_targets_of_deletion, minus_targets_of_deletion, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)\
                                    for edge in edges)
    print("time passed", time()-start)
    knockout_names, results =zip(*packed_results)
    data = pd.DataFrame().from_records(results, index=[0,1], columns = knockout_names[0])
    with open(FT_DIR+name+'_'+PERT_MAP+'.ft','wb') as f:
        pickle.dump(data, f)

Number of threads: 4


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [71]:
print('feature generator inputs:\n raw_matrix (the base network matrix):',type(raw_matrix),'\nedges',type(edges),'\ngene indexes dict of geneid:index of gene in matrix:',type(gene_indexes),'\nmatrix(normalized similarity matrix by number of outgoing edges):',type(matrix),' \nplus/minus_targets_of_deletion: dict. {knockoutgene:list of target positive(?negative) genes}', type(plus_targets_of_deletion))

feature generator inputs:
 raw_matrix (the base network matrix): <class 'scipy.sparse._csr.csr_matrix'> 
edges <class 'list'> 
gene indexes dict of geneid:index of gene in matrix: <class 'dict'> 
matrix(normalized similarity matrix by number of outgoing edges): <class 'scipy.sparse._csr.csr_matrix'>  
plus/minus_targets_of_deletion: dict. {knockoutgene:list of target positive(?negative) genes} <class 'dict'>


In [73]:
print('    returns:\n\
        knockout_names:  list [0,1,knockout1+,knockout2+...knockoutn+,\n\
                               knockout1-,...knockoutn-]\n\
        score_of: list  [edge1ID,edge2ID, score1,....score2n]')

    returns:
        knockout_names:  list [0,1,knockout1+,knockout2+...knockoutn+,
                               knockout1-,...knockoutn-]
        score_of: list  [edge1ID,edge2ID, score1,....score2n]


In [ ]:
for name, edges in zip(names, edges_list):
    print(name)
    start=time()
    knockout_names, results = create_the_features_different_knockouts_iterative(raw_matrix, edges, gene_indexes, matrix, plus_targets_of_deletion,\
                          minus_targets_of_deletion,  num_genes, PROPAGATE_ALPHA, \
                              PROPAGATE_ITERATIONS,PROPAGATE_EPSILON )

    print("time passed", time()-start)
    data = pd.DataFrame().from_records(results, index=[0,1], columns = knockout_names[0])
    with open(FT_DIR+name+'_'+PERT_MAP+'.ft','wb') as f:
        pickle.dump(data, f)

ubinet2
time passed 3267.2936017513275


In [ ]:
data

853836+       855926+       853219+       851858+  \
0      1                                                                
853140 853002  1.899045e-11  3.191363e-11 -5.858731e-11 -5.946693e-11   
       856237 -6.517823e-12 -2.356951e-11 -2.747121e-10 -2.505436e-10   
       851691 -1.216313e-12  2.196772e-11 -8.032611e-12  2.715181e-11   
851485 851810 -2.687803e-10 -4.837991e-10 -1.725504e-10 -1.513739e-10   
       852284 -2.540274e-10 -4.960840e-10 -1.774948e-10 -1.882297e-10   
...                     ...           ...           ...           ...   
854886 852028 -2.271268e-10 -9.528439e-11  1.826659e-11  5.610510e-11   
851900 851900 -8.420989e-11 -8.035088e-11 -2.778743e-10 -2.718255e-10   
853096 850795  3.179609e-10 -5.599058e-11 -4.129779e-10 -4.970562e-10   
851721 856483  4.027867e-11 -5.529062e-11 -5.776926e-10 -6.014080e-10   
854765 853817 -1.418876e-10 -1.512585e-10 -4.019596e-10 -3.217180e-10   

                    853976+       852088+       856476+       850440+  \
0      1                                                                
853140 853002 -1.669091e-11 -6.517051e-11  5.531219e-11 -4.108983e-10   
       856237 -1.184936e-11 -4.309620e-10 -2.709599e-11 -1.140139e-10   
       851691 -3.329214e-12 -3.531687e-10 -1.447521e-10 -7.691483e-10   
851485 851810 -4.493811e-11 -1.879734e-10 -5.736392e-11 -7.850406e-10   
       852284 -2.112286e-11 -2.205321e-10 -8.114605e-11 -9.836039e-10   
...                     ...           ...           ...           ...   
854886 852028 -5.896088e-11 -2.069293e-10  4.914315e-12 -1.046643e-09   
851900 851900 -5.810042e-11 -2.664398e-10 -1.441399e-10 -5.849549e-09   
853096 850795  1.658890e-10 -5.646988e-10 -3.804659e-09 -1.853546e-09   
851721 856483  4.703748e-11 -5.252100e-11 -1.087975e-10 -1.999749e-09   
854765 853817 -2.280760e-11 -5.642487e-10 -1.824170e-10 -1.135128e-08   

                    854034+       854162+  ...       851708-       854874-  \
0      1                                   ...                               
853140 853002  7.935082e-09  7.647116e-11  ... -2.200579e-09 -2.708491e-10   
       856237 -1.264003e-09 -1.606461e-11  ...  5.758614e-08 -5.316960e-10   
       851691 -1.141266e-09  8.602020e-12  ... -6.685432e-09 -7.451806e-10   
851485 851810 -2.418077e-10 -6.407665e-12  ... -1.727301e-09 -1.240970e-10   
       852284 -5.102156e-11  4.246659e-11  ... -1.601214e-09 -9.571169e-11   
...                     ...           ...  ...           ...           ...   
854886 852028 -8.277157e-10 -7.095350e-11  ... -1.004820e-09  3.880410e-11   
851900 851900 -3.686509e-10 -9.020548e-11  ... -9.337626e-10 -1.037782e-10   
853096 850795 -4.622734e-10 -3.297333e-11  ... -1.323070e-09 -6.812506e-11   
851721 856483 -6.007148e-10 -1.141442e-10  ... -6.261284e-10 -1.039974e-10   
854765 853817 -9.370816e-11 -3.590090e-11  ...  1.710481e-09 -2.554244e-07   

                    855318-       854351-       855648-       854087-  \
0      1                                                                
853140 853002 -2.898193e-10 -5.340778e-11 -1.871738e-10 -1.420390e-10   
       856237 -1.249678e-09 -2.594985e-10 -7.159167e-10 -3.300058e-10   
       851691 -1.030654e-09 -1.458210e-10 -4.109426e-10 -3.019428e-10   
851485 851810 -3.506286e-10 -1.213955e-09 -1.249283e-09 -6.526363e-10   
       852284 -4.360288e-10 -1.244771e-09 -8.627952e-10 -4.101192e-10   
...                     ...           ...           ...           ...   
854886 852028 -1.307194e-10 -4.191242e-10 -4.571824e-10 -1.160465e-09   
851900 851900 -8.953398e-10 -3.068385e-10 -7.750139e-10 -6.218965e-10   
853096 850795 -1.605774e-09 -1.161676e-10  5.801797e-10  8.125978e-10   
851721 856483 -6.170164e-10 -4.813416e-10 -3.917747e-10 -3.864497e-10   
854765 853817 -2.877282e-10 -5.060600e-10 -6.598488e-10 -7.959604e-10   

                    852640-       852171-       850787-       856924-  
0      1                                                               
853140 8530

In [27]:
def propagate_gpu(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON, seeds, matrix, gene_indexes, num_genes):
    '''Propagate from seeds inside matrix
    gene indexes is a dictinary mapping gene names to their indexes in matrix
    matrix must be a cupyx.scipy.sparse._csr.csr_matrix'''

    if not type(matrix) == cp.sparse._csr.csr_matrix:
        return ('ERROR! matrix is not a cp.sparse.csr.csr_matrix', type(matrix))

    # matrix = np.array(matrix)
    F_t = cp.zeros(num_genes)
    F_t[[gene_indexes[seed] for seed in seeds if seed in gene_indexes]] = 1
    Y = (1 - PROPAGATE_ALPHA) * F_t

    for _ in range(PROPAGATE_ITERATIONS):
        F_t_1 = F_t  #05 045 2022v add transpose perche adesso matrix e  una np.matrix e bo se no mi da errore
        F_t =  PROPAGATE_ALPHA*matrix.dot(F_t_1) + Y #105 094 2022, rimossa la mia modifica in cui mettevo PROPAGATE_ALPHA* matrix qui (e lhor rimesso nel return della funzione sopra) perchedava errore se matrix e' np.matrix, e non scipy.sparse. cazzo. CAZZO. LOL

        if math.sqrt(cp.linalg.norm(F_t_1 - F_t)) < PROPAGATE_EPSILON:
            break
    return F_t

def defective_matrix_wrapper(raw_matrix, matrix, edge, gene_indexes):
  """Builds the defective PPI matrix for a given row of the feature matrix.
  edge: tuple of geneids of the given edge, representing the feature matrix row"""

  col1, col2, ind1, ind2 = create_defective_columns(raw_matrix, edge, gene_indexes)
  defective_normalized_matrix = defective_norm_matrix_from_columns(matrix, col1, col2,ind1, ind2 )
  return defective_normalized_matrix

def S_score_gpu(defective_normalized_matrix, prop_edge_fwd, edge, source, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
  """Combines row and column scores to get the element score.

  defective_normalized_matrix: row-wise defective normalized matrix. Previously calculated because it is invariant across rows

  prop_edge_fwd: column-wise array of propagation scores from knockout K. Previously calculated because it is invariant across columns

  source: knockout K gene id

  """
  prop_noedge_fwd = propagate_gpu(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON,{source}, defective_normalized_matrix, gene_indexes,num_genes)

  score_of_cell_plus = score(prop_edge_fwd, prop_noedge_fwd,[gene_indexes[x] for x in plus_targets_of_deletion[source]],edge=edge)
  score_of_cell_minus = score(prop_edge_fwd, prop_noedge_fwd,[gene_indexes[x] for x in minus_targets_of_deletion[source]],edge=edge)

  return score_of_cell_plus, score_of_cell_minus

def create_the_features_different_knockouts_optimized_gpu(edges, plus_targets_of_deletion, minus_targets_of_deletion, raw_matrix, matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
  """Fills a 2d feature matrix of SIGNAL features. does less steps than the old create_the_features_different_knockouts_iterative"""

  # Calculate row and column scores on the GPU
  row_names=edges
  col_names=list(plus_targets_of_deletion.keys())+list(minus_targets_of_deletion.keys()) #they are the same keys repeated
  row_defective_matrices = [defective_matrix_wrapper(raw_matrix, matrix, i, gene_indexes) for i in row_names] # cupy cannot hold a list of sparse matrices object, so we keep it as a list
  col_propagation_scores = [propagate_gpu(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON, {j}, matrix, gene_indexes, num_genes) for j in col_names]
  # Create an empty matrix on the GPU
  features_matrix = cp.zeros((len(row_names), len(col_names)), dtype=np.float32)

  # Fill the matrix element-wise using broadcasting and the combine_scores function
  for i, edge in enumerate(edges):
    # fill 2 column indexes at a time
    for j, source in enumerate(col_names[:int(len(col_names)/2)]):
      features_matrix[i, [j,j+int(len(col_names)/2)]] = S_score_gpu(row_defective_matrices[i], col_propagation_scores[j], edge, source, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)

  # Transfer the filled matrix back to the CPU as a NumPy array
  return cp.asnumpy(features_matrix)




In [28]:
# convert numpy and scipy objects to cupy objects
cp_raw_matrix = cp.sparse._csr.csr_matrix(raw_matrix)
cp_matrix = cp.sparse._csr.csr_matrix(matrix)


In [31]:
for name, edges in zip(names, edges_list):
  start=time()
  feature_matrix=create_the_features_different_knockouts_optimized_gpu(edges[:5], plus_targets_of_deletion, minus_targets_of_deletion, cp_raw_matrix, cp_matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)

  pluskeys=[str(source)+'+' for source in plus_targets_of_deletion.keys()]
  minuskeys=[str(source)+'-' for source in minus_targets_of_deletion.keys()]
  datagpu =  pd.DataFrame(feature_matrix, index=pd.MultiIndex.from_tuples(edges[:5], names=[0,1]), columns = pluskeys+minuskeys)
  print("time passed", time()-start)
datagpu

time passed 19.165487051010132


853836+       855926+       853219+       851858+  \
0      1                                                                
853140 853002  1.899045e-11  3.191363e-11 -5.858732e-11 -5.946693e-11   
       856237 -6.517824e-12 -2.356951e-11 -2.747121e-10 -2.505436e-10   
       851691 -1.216313e-12  2.196772e-11 -8.032611e-12  2.715181e-11   
851485 851810 -2.687803e-10 -4.837991e-10 -1.725504e-10 -1.513739e-10   
       852284 -2.540274e-10 -4.960840e-10 -1.774948e-10 -1.882297e-10   

                    853976+       852088+       856476+       850440+  \
0      1                                                                
853140 853002 -1.669091e-11 -6.517051e-11  5.531219e-11 -4.108983e-10   
       856237 -1.184936e-11 -4.309620e-10 -2.709599e-11 -1.140139e-10   
       851691 -3.329214e-12 -3.531687e-10 -1.447521e-10 -7.691483e-10   
851485 851810 -4.493811e-11 -1.879734e-10 -5.736392e-11 -7.850406e-10   
       852284 -2.112286e-11 -2.205321e-10 -8.114605e-11 -9.836039e-10   

                    854034+       854162+  ...       851708-       854874-  \
0      1                                   ...                               
853140 853002  7.935082e-09  7.647116e-11  ... -2.200579e-09 -2.708492e-10   
       856237 -1.264003e-09 -1.606461e-11  ...  5.758614e-08 -5.316960e-10   
       851691 -1.141266e-09  8.602020e-12  ... -6.685432e-09 -7.451806e-10   
851485 851810 -2.418077e-10 -6.407665e-12  ... -1.727301e-09 -1.240970e-10   
       852284 -5.102156e-11  4.246659e-11  ... -1.601214e-09 -9.571169e-11   

                    855318-       854351-       855648-       854087-  \
0      1                                                                
853140 853002 -2.898193e-10 -5.340778e-11 -1.871738e-10 -1.420390e-10   
       856237 -1.249678e-09 -2.594985e-10 -7.159167e-10 -3.300058e-10   
       851691 -1.030654e-09 -1.458210e-10 -4.109426e-10 -3.019428e-10   
851485 851810 -3.506286e-10 -1.213955e-09 -1.249283e-09 -6.526363e-10   
       852284 -4.360288e-10 -1.244771e-09 -8.627952e-10 -4.101192e-10   

                    852640-       852171-       850787-       856924-  
0      1                                                               
853140 853002 -1.908892e-10 -1.251570e-10 -1.238575e-09 -7.709518e-10  
       856237 -7.301277e-10 -2.947551e-10 -2.626421e-09  2.005869e-10  
       851691 -4.190999e-10 -2.651931e-10 -2.389253e-09 -2.411320e-10  
851485 851810 -5.993045e-09 -6.335565e-10 -1.167252e-10  4.577783e-10  
       852284 -4.138990e-09 -3.991362e-10 -2.331051e-10 -1.931467e-09  

[5 rows x 472 columns]

In [26]:
def defective_matrix_wrapper(raw_matrix, matrix, edge, gene_indexes):
  """Builds the defective PPI matrix for a given row of the feature matrix.
  edge: tuple of geneids of the given edge, representing the feature matrix row"""

  col1, col2, ind1, ind2 = create_defective_columns(raw_matrix, edge, gene_indexes)
  defective_normalized_matrix = defective_norm_matrix_from_columns(matrix, col1, col2,ind1, ind2 )

  return defective_normalized_matrix

def S_score(defective_normalized_matrix, prop_edge_fwd, edge, source, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
  """Combines row and column scores to get the element score.

  defective_normalized_matrix: row-wise defective normalized matrix. Previously calculated because it is invariant across rows

  prop_edge_fwd: column-wise array of propagation scores from knockout K. Previously calculated because it is invariant across columns

  source: knockout K gene id

  """
  prop_noedge_fwd = propagate(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON,{source}, defective_normalized_matrix, gene_indexes,num_genes)

  score_of_cell_plus = score(prop_edge_fwd, prop_noedge_fwd,[gene_indexes[x] for x in plus_targets_of_deletion[source]],edge=edge)
  score_of_cell_minus = score(prop_edge_fwd, prop_noedge_fwd,[gene_indexes[x] for x in minus_targets_of_deletion[source]],edge=edge)

  return score_of_cell_plus, score_of_cell_minus

def create_the_features_different_knockouts_iterative_optimized(edges, plus_targets_of_deletion, minus_targets_of_deletion, raw_matrix, matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
  """Fills a 2d feature matrix of SIGNAL features. does less steps than the old create_the_features_different_knockouts_iterative"""

  # Calculate row and column scores on the GPU
  row_names=edges
  col_names=list(plus_targets_of_deletion.keys())+list(minus_targets_of_deletion.keys()) #they are the same keys repeated
  row_defective_matrices = np.array([defective_matrix_wrapper(raw_matrix, matrix, i, gene_indexes) for i in row_names])
  col_propagation_scores = np.array([propagate(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON, {j}, matrix, gene_indexes, num_genes) for j in col_names])
  # Create an empty matrix on the GPU
  features_matrix = np.zeros((len(row_names), len(col_names)), dtype=np.float32)

  # Fill the matrix element-wise using broadcasting and the combine_scores function
  for i, edge in enumerate(edges):
    # fill 2 column indexes at a time
    for j, source in enumerate(col_names[:int(len(col_names)/2)]):
      features_matrix[i, [j,j+int(len(col_names)/2)]] = S_score(row_defective_matrices[i], col_propagation_scores[j], edge, source, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)

  # Transfer the filled matrix back to the CPU as a NumPy array
  return features_matrix#cp.asnumpy(features_matrix)

for name, edges in zip(names, edges_list):
  start=time()
  feature_matrix=create_the_features_different_knockouts_iterative_optimized(edges[:5], plus_targets_of_deletion, minus_targets_of_deletion, raw_matrix, matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)

  pluskeys=[str(source)+'+' for source in plus_targets_of_deletion.keys()]
  minuskeys=[str(source)+'-' for source in minus_targets_of_deletion.keys()]
  dataiter =  pd.DataFrame(feature_matrix, index=pd.MultiIndex.from_tuples(edges[:5], names=[0,1]), columns = pluskeys+minuskeys)
  print("time passed", time()-start)
dataiter

time passed 27.317442178726196


853836+       855926+       853219+       851858+  \
0      1                                                                
853140 853002  1.899045e-11  3.191363e-11 -5.858732e-11 -5.946693e-11   
       856237 -6.517824e-12 -2.356951e-11 -2.747121e-10 -2.505436e-10   
       851691 -1.216313e-12  2.196772e-11 -8.032611e-12  2.715181e-11   
851485 851810 -2.687803e-10 -4.837991e-10 -1.725504e-10 -1.513739e-10   
       852284 -2.540274e-10 -4.960840e-10 -1.774948e-10 -1.882297e-10   

                    853976+       852088+       856476+       850440+  \
0      1                                                                
853140 853002 -1.669091e-11 -6.517051e-11  5.531219e-11 -4.108983e-10   
       856237 -1.184936e-11 -4.309620e-10 -2.709599e-11 -1.140139e-10   
       851691 -3.329214e-12 -3.531687e-10 -1.447521e-10 -7.691483e-10   
851485 851810 -4.493811e-11 -1.879734e-10 -5.736392e-11 -7.850406e-10   
       852284 -2.112286e-11 -2.205321e-10 -8.114605e-11 -9.836039e-10   

                    854034+       854162+  ...       851708-       854874-  \
0      1                                   ...                               
853140 853002  7.935082e-09  7.647116e-11  ... -2.200579e-09 -2.708491e-10   
       856237 -1.264003e-09 -1.606461e-11  ...  5.758614e-08 -5.316960e-10   
       851691 -1.141266e-09  8.602020e-12  ... -6.685432e-09 -7.451806e-10   
851485 851810 -2.418077e-10 -6.407665e-12  ... -1.727301e-09 -1.240970e-10   
       852284 -5.102156e-11  4.246659e-11  ... -1.601214e-09 -9.571169e-11   

                    855318-       854351-       855648-       854087-  \
0      1                                                                
853140 853002 -2.898193e-10 -5.340778e-11 -1.871738e-10 -1.420390e-10   
       856237 -1.249678e-09 -2.594985e-10 -7.159167e-10 -3.300058e-10   
       851691 -1.030654e-09 -1.458210e-10 -4.109426e-10 -3.019428e-10   
851485 851810 -3.506286e-10 -1.213955e-09 -1.249283e-09 -6.526363e-10   
       852284 -4.360288e-10 -1.244771e-09 -8.627952e-10 -4.101192e-10   

                    852640-       852171-       850787-       856924-  
0      1                                                               
853140 853002 -1.908892e-10 -1.251570e-10 -1.238575e-09 -7.709518e-10  
       856237 -7.301277e-10 -2.947551e-10 -2.626421e-09  2.005869e-10  
       851691 -4.190999e-10 -2.651931e-10 -2.389253e-09 -2.411320e-10  
851485 851810 -5.993045e-09 -6.335565e-10 -1.167252e-10  4.577783e-10  
       852284 -4.138990e-09 -3.991362e-10 -2.331051e-10 -1.931467e-09  

[5 rows x 472 columns]

In [154]:
data

853836+       855926+       853219+       851858+  \
0      1                                                                
853140 853002  1.899045e-11  3.191363e-11 -5.858731e-11 -5.946693e-11   
       856237 -6.517823e-12 -2.356951e-11 -2.747121e-10 -2.505436e-10   
       851691 -1.216313e-12  2.196772e-11 -8.032611e-12  2.715181e-11   
851485 851810 -2.687803e-10 -4.837991e-10 -1.725504e-10 -1.513739e-10   
       852284 -2.540274e-10 -4.960840e-10 -1.774948e-10 -1.882297e-10   

                    853976+       852088+       856476+       850440+  \
0      1                                                                
853140 853002 -1.669091e-11 -6.517051e-11  5.531219e-11 -4.108983e-10   
       856237 -1.184936e-11 -4.309620e-10 -2.709599e-11 -1.140139e-10   
       851691 -3.329214e-12 -3.531687e-10 -1.447521e-10 -7.691483e-10   
851485 851810 -4.493811e-11 -1.879734e-10 -5.736392e-11 -7.850406e-10   
       852284 -2.112286e-11 -2.205321e-10 -8.114605e-11 -9.836039e-10   

                    854034+       854162+  ...       851708-       854874-  \
0      1                                   ...                               
853140 853002  7.935082e-09  7.647116e-11  ... -2.200579e-09 -2.708491e-10   
       856237 -1.264003e-09 -1.606461e-11  ...  5.758614e-08 -5.316960e-10   
       851691 -1.141266e-09  8.602020e-12  ... -6.685432e-09 -7.451806e-10   
851485 851810 -2.418077e-10 -6.407665e-12  ... -1.727301e-09 -1.240970e-10   
       852284 -5.102156e-11  4.246659e-11  ... -1.601214e-09 -9.571169e-11   

                    855318-       854351-       855648-       854087-  \
0      1                                                                
853140 853002 -2.898193e-10 -5.340778e-11 -1.871738e-10 -1.420390e-10   
       856237 -1.249678e-09 -2.594985e-10 -7.159167e-10 -3.300058e-10   
       851691 -1.030654e-09 -1.458210e-10 -4.109426e-10 -3.019428e-10   
851485 851810 -3.506286e-10 -1.213955e-09 -1.249283e-09 -6.526363e-10   
       852284 -4.360288e-10 -1.244771e-09 -8.627952e-10 -4.101192e-10   

                    852640-       852171-       850787-       856924-  
0      1                                                               
853140 853002 -1.908892e-10 -1.251570e-10 -1.238575e-09 -7.709518e-10  
       856237 -7.301277e-10 -2.947551e-10 -2.626421e-09  2.005869e-10  
       851691 -4.190999e-10 -2.651931e-10 -2.389253e-09 -2.411320e-10  
851485 851810 -5.993045e-09 -6.335566e-10 -1.167252e-10  4.577783e-10  
       852284 -4.138990e-09 -3.991362e-10 -2.331051e-10 -1.931467e-09  

[5 rows x 472 columns]

In [25]:
def create_the_features_different_knockouts_parallel_optimized(N_JOBS, edges, plus_targets_of_deletion, minus_targets_of_deletion, raw_matrix, matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
  """Fills a 2d feature matrix of SIGNAL features. does less steps than the old create_the_features_different_knockouts_iterative"""

  # Calculate row and column scores on the GPU
  row_names=edges
  col_names=list(plus_targets_of_deletion.keys())+list(minus_targets_of_deletion.keys()) #they are the same keys repeated
  row_defective_matrices = np.array([defective_matrix_wrapper(raw_matrix, matrix, i, gene_indexes) for i in row_names])
  col_propagation_scores = np.array([propagate(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON, {j}, matrix, gene_indexes, num_genes) for j in col_names])


  # Fill the matrix element-wise using broadcasting and the combine_scores function

  def fill_row(i,edge,row_defective_matrices, col_propagation_scores, col_names, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
    row_plus, row_minus= [], []
    for j, source in enumerate(col_names[:int(len(col_names)/2)]):
      score_of_cell_plus, score_of_cell_minus = S_score(row_defective_matrices[i], col_propagation_scores[j], edge, source, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)
      row_plus.append(score_of_cell_plus)
      row_minus.append(score_of_cell_minus)
    return row_plus+ row_minus


  features_matrix = Parallel(n_jobs=N_JOBS)(delayed(fill_row)\
                                (i,edge,row_defective_matrices, col_propagation_scores, col_names, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)\
                                    for i, edge in enumerate(edges))


  # Transfer the filled matrix back to the CPU as a NumPy array
  return features_matrix



In [47]:
for name, edges in zip(names, edges_list):
  start=time()
  feature_matrix=create_the_features_different_knockouts_parallel_optimized(N_JOBS,edges[:5], plus_targets_of_deletion, minus_targets_of_deletion, raw_matrix, matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)

  pluskeys=[str(source)+'+' for source in plus_targets_of_deletion.keys()]
  minuskeys=[str(source)+'-' for source in minus_targets_of_deletion.keys()]
  dataparal =  pd.DataFrame(feature_matrix, index=pd.MultiIndex.from_tuples(edges[:5], names=[0,1]), columns = pluskeys+minuskeys)
  print("time passed", time()-start)
dataparal

time passed 25.496645212173462


853836+       855926+       853219+       851858+  \
0      1                                                                
853140 853002  1.899045e-11  3.191363e-11 -5.858731e-11 -5.946693e-11   
       856237 -6.517823e-12 -2.356951e-11 -2.747121e-10 -2.505436e-10   
       851691 -1.216313e-12  2.196772e-11 -8.032611e-12  2.715181e-11   
851485 851810 -2.687803e-10 -4.837991e-10 -1.725504e-10 -1.513739e-10   
       852284 -2.540274e-10 -4.960840e-10 -1.774948e-10 -1.882297e-10   

                    853976+       852088+       856476+       850440+  \
0      1                                                                
853140 853002 -1.669091e-11 -6.517051e-11  5.531219e-11 -4.108983e-10   
       856237 -1.184936e-11 -4.309620e-10 -2.709599e-11 -1.140139e-10   
       851691 -3.329214e-12 -3.531687e-10 -1.447521e-10 -7.691483e-10   
851485 851810 -4.493811e-11 -1.879734e-10 -5.736392e-11 -7.850406e-10   
       852284 -2.112286e-11 -2.205321e-10 -8.114605e-11 -9.836039e-10   

                    854034+       854162+  ...       851708-       854874-  \
0      1                                   ...                               
853140 853002  7.935082e-09  7.647116e-11  ... -2.200579e-09 -2.708491e-10   
       856237 -1.264003e-09 -1.606461e-11  ...  5.758614e-08 -5.316960e-10   
       851691 -1.141266e-09  8.602020e-12  ... -6.685432e-09 -7.451806e-10   
851485 851810 -2.418077e-10 -6.407665e-12  ... -1.727301e-09 -1.240970e-10   
       852284 -5.102156e-11  4.246659e-11  ... -1.601214e-09 -9.571169e-11   

                    855318-       854351-       855648-       854087-  \
0      1                                                                
853140 853002 -2.898193e-10 -5.340778e-11 -1.871738e-10 -1.420390e-10   
       856237 -1.249678e-09 -2.594985e-10 -7.159167e-10 -3.300058e-10   
       851691 -1.030654e-09 -1.458210e-10 -4.109426e-10 -3.019428e-10   
851485 851810 -3.506286e-10 -1.213955e-09 -1.249283e-09 -6.526363e-10   
       852284 -4.360288e-10 -1.244771e-09 -8.627952e-10 -4.101192e-10   

                    852640-       852171-       850787-       856924-  
0      1                                                               
853140 853002 -1.908892e-10 -1.251570e-10 -1.238575e-09 -7.709518e-10  
       856237 -7.301277e-10 -2.947551e-10 -2.626421e-09  2.005869e-10  
       851691 -4.190999e-10 -2.651931e-10 -2.389253e-09 -2.411320e-10  
851485 851810 -5.993045e-09 -6.335566e-10 -1.167252e-10  4.577783e-10  
       852284 -4.138990e-09 -3.991362e-10 -2.331051e-10 -1.931467e-09  

[5 rows x 472 columns]

In [46]:
def create_the_features_different_knockouts_parallel_optimized_gpu(N_JOBS, edges, plus_targets_of_deletion, minus_targets_of_deletion, raw_matrix, matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
  """Fills a 2d feature matrix of SIGNAL features. does less steps than the old create_the_features_different_knockouts_iterative"""

  # Calculate row and column scores on the GPU
  row_names=edges
  col_names=list(plus_targets_of_deletion.keys())+list(minus_targets_of_deletion.keys()) #they are the same keys repeated
  row_defective_matrices = [defective_matrix_wrapper(raw_matrix, matrix, i, gene_indexes) for i in row_names] # cupy cannot hold a list of sparse matrices object, so we keep it as a list
  col_propagation_scores = [propagate_gpu(PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON, {j}, matrix, gene_indexes, num_genes) for j in col_names]

  # Fill the matrix element-wise using broadcasting and the combine_scores function

  def fill_row(i,edge,row_defective_matrices, col_propagation_scores, col_names, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON):
    row_plus, row_minus= [], []
    for j, source in enumerate(col_names[:int(len(col_names)/2)]):
      score_of_cell_plus, score_of_cell_minus = S_score_gpu(row_defective_matrices[i], col_propagation_scores[j], edge, source, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)
      row_plus.append(cp.asnumpy(score_of_cell_plus))
      row_minus.append(cp.asnumpy(score_of_cell_minus))
    return row_plus+ row_minus


  features_matrix = Parallel(n_jobs=N_JOBS)(delayed(fill_row)\
                                (i,edge,row_defective_matrices, col_propagation_scores, col_names, plus_targets_of_deletion, minus_targets_of_deletion, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)\
                                    for i, edge in enumerate(edges))


  return features_matrix

In [41]:
for name, edges in zip(names, edges_list):
  start=time()
  feature_matrix=create_the_features_different_knockouts_parallel_optimized_gpu(N_JOBS, edges[:5], plus_targets_of_deletion, minus_targets_of_deletion, cp_raw_matrix, cp_matrix, gene_indexes, num_genes, PROPAGATE_ALPHA, PROPAGATE_ITERATIONS, PROPAGATE_EPSILON)

  pluskeys=[str(source)+'+' for source in plus_targets_of_deletion.keys()]
  minuskeys=[str(source)+'-' for source in minus_targets_of_deletion.keys()]
  dataparal =  pd.DataFrame(feature_matrix, index=pd.MultiIndex.from_tuples(edges[:5], names=[0,1]), columns = pluskeys+minuskeys)
  print("time passed", time()-start)
dataparal

non funge
funge
time passed 20.862287282943726


853836+                  855926+  \
0      1                                                          
853140 853002   1.8990453876316718e-11    3.191362954880605e-11   
       856237   -6.517823461531973e-12  -2.3569512589875037e-11   
       851691  -1.2163129559799885e-12   2.1967718080397785e-11   
851485 851810    -2.68780284124983e-10   -4.837990992557757e-10   
       852284  -2.5402736724639626e-10   -4.960840049128112e-10   

                               853219+                  851858+  \
0      1                                                          
853140 853002    -5.85873147244335e-11   -5.946692949774714e-11   
       856237  -2.7471206074932596e-10   -2.505436001768536e-10   
       851691    -8.03261106176352e-12     2.71518122220685e-11   
851485 851810   -1.725503673140396e-10  -1.5137387903524393e-10   
       852284  -1.7749477695541317e-10  -1.8822966729988187e-10   

                               853976+                  852088+  \
0      1                                                          
853140 853002   -1.669091275273134e-11   -6.517051128081524e-11   
       856237  -1.1849359018403956e-11   -4.309620202274712e-10   
       851691   -3.329213625632532e-12   -3.531686603659213e-10   
851485 851810   -4.493811271145639e-11  -1.8797335472290223e-10   
       852284  -2.1122857460945067e-11  -2.2053211686678563e-10   

                               856476+                  850440+  \
0      1                                                          
853140 853002    5.531219185192389e-11  -4.1089832653779993e-10   
       856237  -2.7095988545474296e-11   -1.140139374388736e-10   
       851691  -1.4475212361638664e-10    -7.69148275943382e-10   
851485 851810    -5.73639167776093e-11   -7.850406364278076e-10   
       852284   -8.114604970201264e-11   -9.836038502275253e-10   

                               854034+                  854162+  ...  \
0      1                                                         ...   
853140 853002    7.935082332741386e-09    7.647116178508765e-11  ...   
       856237  -1.2640033286111583e-09  -1.6064614287601892e-11  ...   
       851691   -1.141265934129274e-09     8.60201973128423e-12  ...   
851485 851810   -2.418077196622185e-10  -6.4076652495662035e-12  ...   
       852284   -5.102156061691049e-11    4.246658647935684e-11  ...   

                               851708-                 854874-  \
0      1                                                         
853140 853002   -2.200579107825007e-09  -2.708491764202847e-10   
       856237   5.7586140023158094e-08   -5.31696020544814e-10   
       851691  -6.6854316249642665e-09  -7.451805839053804e-10   
851485 851810  -1.7273014407681586e-09  -1.240970104454675e-10   
       852284  -1.6012136616616601e-09  -9.571168857469559e-11   

                               855318-                  854351-  \
0      1                                                          
853140 853002   -2.898192796862986e-10  -5.3407778199954237e-11   
       856237  -1.2496783052817761e-09  -2.5949850601669766e-10   
       851691  -1.0306542841398425e-09   -1.458210219240641e-10   
851485 851810    -3.50628609480097e-10  -1.2139554361745297e-09   
       852284   -4.360288241578303e-10  -1.2447705088902694e-09   

                               855648-                 854087-  \
0      1                                                         
853140 853002   -1.871737668205026e-10  -1.420389918331226e-10   
       856237   -7.159166799344157e-10  -3.300057916029565e-10   
       851691    -4.10942637801533e-10  -3.019428010447136e-10   
851485 851810  -1.2492832950148758e-09  -6.526362615955138e-10   
       852284   -8.627951767208215e-10  -4.101192122983581e-10   

                               852640-                  852171-  \
0      1                                                          
853140 853002  -1.9088918633443624e-10  -1.2515704039343988e-10   
       856237   -7.301276966321327e-10    -2.9475